# RASMapper Layer Generator - Mapping Time Intervals for Raster Animations
Author: William Katzenmeyer, P.E., C.F.M.

This RASMapper Layer Generator is a Python script that automates the process of generating time-based map layers for raster animations in HEC-RAS. It modifies the `.rasmap` file to include map layers at specified time intervals, allowing for dynamic visualization of flood progression over time using external packages.

## Features

- Automatically finds the `.rasmap` file in the specified project directory
- Extracts the HDF file path and plan name from the `.rasmap` file
- Generates time stamps based on the specified mapping period start, end, and interval
- Extracts profile indices corresponding to the generated time stamps from the HDF file
- Inserts new map layer entries into the `.rasmap` file for each time stamp and profile index
- Creates a backup of the original `.rasmap` file before modifying it
- Modifies the associated plan file to prepare the project for floodplain mapping

## Prerequisites

- Python 3.x
- Required Python packages:
 - `h5py`: Install using `pip install h5py`

## Usage

1. Set the following user-defined variables in the script:
  - `ProjectDirectoryPath`: The path to the HEC-RAS project directory
  - `ResultsLayerName`: The name of the results layer in the `.rasmap` file
  - `MappingPeriodStart`: The start date and time of the mapping period (format: "DDMMMYYYY HH:MM:SS")
  - `MappingPeriodEnd`: The end date and time of the mapping period (format: "DDMMMYYYY HH:MM:SS")
  - `MappingInterval`: The interval between each map layer, in hours
  - `TerrainName`: The name of the terrain used for mapping

2. Run the script.

3. The script will perform the following actions:
  - Find the `.rasmap` file in the specified project directory
  - Extract the HDF file path and plan name from the `.rasmap` file
  - Generate time stamps based on the specified mapping period and interval
  - Extract profile indices corresponding to the generated time stamps from the HDF file
  - Insert new map layer entries into the `.rasmap` file for each time stamp and profile index
  - Create a backup of the original `.rasmap` file with a `.backupX` extension (where `X` is an incrementing number)
  - Modify the associated plan file to prepare the project for floodplain mapping

4. After the script completes, the modified `.rasmap` file will include the new map layers at the specified time intervals, ready for raster animation.

## Troubleshooting

- If the script fails to find the `.rasmap` file or the HDF file path, ensure that the `ProjectDirectoryPath` and `ResultsLayerName` variables are correctly set.
- If the script fails to extract profile indices, ensure that the generated time stamps match the time stamps in the HDF file.
- If the script fails to insert layer entries, ensure that the XML structure of the `.rasmap` file is valid and contains a 'Layer' section with the specified `ResultsLayerName`.

## Disclaimer

This script modifies the `.rasmap` and plan files in the HEC-RAS project. It is recommended to create a backup of the entire project before running the script. The script author and organization are not responsible for any data loss or damage caused by the use of this script. Use at your own risk.

For any further assistance or questions, please contact the script author or refer to the HEC-RAS documentation.



In [1]:
# Install H5Py in your environment by uncommenting the line below
#!pip install h5py
# uncomment to install in your environment

In [2]:
import os
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import h5py
import re
import shutil

# User-defined variables
ProjectDirectoryPath = r"C:\WMK_Working\WF_WestForkCalcasieu\RAS1D_template"

# Mapping Period Start and End should be in the format "DDMMMYYYY HH:MM:SS"

ResultsLayerName = "DELTA2020"
MappingPeriodStart = "28SEP2020 00:00:00"
MappingPeriodEnd = "05OCT2020 22:00:00"
MappingInterval = 6 # in hours
TerrainName = "WF_Terrain_Final" 







# Function to create an XML layer entry
def create_layer_entry(profile_index, time_stamp, terrain_name, results_layer_name):
    layer_entry_xml = f'''
        <Layer Name="WSE" Type="RASResultsMap" Checked="True" Filename=".\\{results_layer_name}\\WSE ({time_stamp.replace(':', ' ').replace(' ', '_')}).vrt">
            <MapParameters MapType="elevation" LayerName="MaxWSE_{time_stamp.replace(' ', '_')}" OutputMode="Stored Current Terrain" StoredFilename=".\\{results_layer_name}\\WSE ({time_stamp.replace(':', ' ').replace(' ', '_')}).vrt" Terrain="{terrain_name}" ProfileIndex="{profile_index}" ProfileName="{time_stamp}" ArrivalDepth="0" />
        </Layer>
    '''
    return ET.fromstring(layer_entry_xml.strip())


# Function to find the first .rasmap file in the given directory
def find_rasmap_file(directory_path):
    for file in os.listdir(directory_path):
        if file.endswith(".rasmap"):
            return os.path.join(directory_path, file)
    raise FileNotFoundError("No .rasmap file found in the directory.")

# Function to extract the HDF file path and PlanName from the .rasmap file
def ExtractHDFFilePathAndPlanNameFromRasmap(rasmap_file_path, results_layer_name, project_directory_path):
    tree = ET.parse(rasmap_file_path)
    root = tree.getroot()
    for layer in root.findall(".//Layer"):
        if layer.get('Name') == results_layer_name and layer.get('Type') == "RASResults":
            filename = layer.get('Filename').lstrip(".\\")
            print("HDF filename: ", filename)
            # Plan name is the name of the HDF file without the hdf extension
            plan_name = os.path.splitext(filename)[0]
            print("Plan name: ", plan_name)
            return os.path.join(project_directory_path, filename), plan_name
    
    raise FileNotFoundError(f"No HDF file path found for layer {results_layer_name} in {rasmap_file_path}")

# Function to generate time stamps
def GenerateTimeStamps(Start, End, IntervalHours):
    StartTime = datetime.strptime(Start, "%d%b%Y %H:%M:%S").replace(tzinfo=None)
    EndTime = datetime.strptime(End, "%d%b%Y %H:%M:%S").replace(tzinfo=None)
    TimeStamps = []
    while StartTime <= EndTime:
        TimeStamps.append(StartTime.strftime("%d%b%Y %H:%M:%S").upper())
        StartTime += timedelta(hours=IntervalHours)
    return TimeStamps

# Function to extract profile indices from the HDF file
def ExtractProfileIndices(HDFFilePath, TimeStamps):
    with h5py.File(HDFFilePath, 'r') as HDFFile:
        DatasetPath = '/Results/Unsteady/Output/Output Blocks/DSS Profile Output/Unsteady Time Series/Time Date Stamp'
        TimeDateStampDataset = HDFFile[DatasetPath]
        TimeDateStamps = [t.decode('ascii') for t in TimeDateStampDataset[:]]
        ProfileIndices = {ts: TimeDateStamps.index(ts) for ts in TimeStamps if ts in TimeDateStamps}
        MissingStamps = [ts for ts in TimeStamps if ts not in TimeDateStamps]
        if MissingStamps:
            raise ValueError(f"Time stamps {MissingStamps} not found in the HDF dataset.")
    return ProfileIndices
# Function to insert layer entries into the XML root
def insert_layer_entries(xml_root, time_stamp_profile_index_dict, terrain_name, results_layer_name):
    results_layer = xml_root.find(f".//Layer[@Name='{results_layer_name}']")
    if results_layer is None:
        raise ValueError(f"The XML structure does not contain a 'Layer' section with Name '{results_layer_name}'.")

    existing_layers = results_layer.findall(".//Layer")
    existing_layer_signatures = {f"{layer.get('Filename')}_{layer.find('.//MapParameters').get('ProfileIndex')}" for layer in existing_layers if layer.find('.//MapParameters') is not None}

    for time_stamp, profile_index in time_stamp_profile_index_dict.items():
        layer_entry_element = create_layer_entry(profile_index, time_stamp, terrain_name, results_layer_name)
        layer_signature = f"{layer_entry_element.get('Filename')}_{layer_entry_element.find('.//MapParameters').get('ProfileIndex')}"
        
        if layer_signature not in existing_layer_signatures:
            results_layer.append(layer_entry_element)
        else:
            print(f"Error: Layer entry for timestamp {time_stamp} with profile index {profile_index} already exists and was not overwritten.")
            print(f"Layer not inserted: {ET.tostring(layer_entry_element, encoding='unicode')}")
    return xml_root

# Main execution
try:
    print("Finding rasmap file path...")
    rasmap_file_path = find_rasmap_file(ProjectDirectoryPath)
    print("Extracting HDF file path and plan name from rasmap...")
    HDFFilePath, PlanName = ExtractHDFFilePathAndPlanNameFromRasmap(rasmap_file_path, ResultsLayerName, ProjectDirectoryPath)
    print("Generating time stamps...")
    GeneratedTimeStamps = GenerateTimeStamps(MappingPeriodStart, MappingPeriodEnd, MappingInterval)
    print("Extracting profile indices...")
    profile_indices = ExtractProfileIndices(HDFFilePath, GeneratedTimeStamps)
    print("Parsing rasmap file...")
    tree = ET.parse(rasmap_file_path)
    root = tree.getroot()
    print("Inserting layer entries...")
    modified_xml_root = insert_layer_entries(root, profile_indices, TerrainName, ResultsLayerName)
    print("Writing modified .rasmap file...")
    tree = ET.ElementTree(modified_xml_root)
    tree.write(rasmap_file_path)  # Assuming new_rasmap_file_path should be rasmap_file_path for overwrite
    print(f"Modified .rasmap file written to {rasmap_file_path}")
except Exception as e:
    print(e)

def CreateBackupAndModifyPlan(PlanName, ProjectDirectoryPath):
    PlanFilePath = os.path.join(ProjectDirectoryPath, PlanName)

    # Check if the plan file exists
    if not os.path.exists(PlanFilePath):
        print(f"The file {PlanName} does not exist in {ProjectDirectoryPath}")
        return

    # Creating a backup
    BackupIndex = 0
    BackupFilePath = PlanFilePath + '.backup'
    while os.path.exists(BackupFilePath):
        BackupFilePath = PlanFilePath + f'.backup{BackupIndex}'
        BackupIndex += 1

    shutil.copy(PlanFilePath, BackupFilePath)
    print(f"Backup created: {BackupFilePath}")

    # Read the contents of the original file
    with open(PlanFilePath, 'r') as file:
        lines = file.readlines()

    # Modify specific lines
    ModifiedLines = []
    for line in lines:
        if 'Run HTab=' in line:
            line = 'Run HTab= 0 \n'
        elif 'Run UNet=' in line:
            line = 'Run UNet= 0 \n'
        ModifiedLines.append(line)

    # Write the modified content back to the file
    with open(PlanFilePath, 'w') as file:
        file.writelines(ModifiedLines)

    print(f"Modifications applied to {PlanName}")

# Call the function
CreateBackupAndModifyPlan(PlanName, ProjectDirectoryPath)



Finding rasmap file path...
Extracting HDF file path and plan name from rasmap...
HDF filename:  WF_WestForkCalcasieu.p01.hdf
Plan name:  WF_WestForkCalcasieu.p01
Generating time stamps...
Extracting profile indices...
Parsing rasmap file...
Inserting layer entries...
Writing modified .rasmap file...
Modified .rasmap file written to C:\WMK_Working\WF_WestForkCalcasieu\RAS1D_template\WF_WestForkCalcasieu.rasmap
Backup created: C:\WMK_Working\WF_WestForkCalcasieu\RAS1D_template\WF_WestForkCalcasieu.p01.backup8
Modifications applied to WF_WestForkCalcasieu.p01
